In [7]:
import pandas as pd
import numpy as np
import random
from faker import Faker

fake = Faker()
np.random.seed(42)
random.seed(42)

# Constants
sectors = ['Technology', 'Finance', 'Healthcare', 'Energy', 'Retail', 'Manufacturing', 'Telecommunications', 'Automotive']
countries = ['USA', 'Germany', 'India', 'China', 'Brazil', 'UK', 'Canada', 'Australia', 'Japan', 'South Korea']
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
years = list(range(2020, 2025))
periods = [(y, q) for y in years for q in quarters]

# 1. Companies Table
num_companies = 500
company_ids = [f"CMP{str(i).zfill(4)}" for i in range(1, num_companies+1)]
companies = pd.DataFrame({
    "CompanyID": company_ids,
    "CompanyName": [fake.company() for _ in range(num_companies)],
    "Sector": [random.choice(sectors) for _ in range(num_companies)],
    "Country": [random.choice(countries) for _ in range(num_companies)],
    "FoundedYear": [random.randint(1950, 2018) for _ in range(num_companies)]
})

# 2. TimePeriods Table
time_periods = pd.DataFrame([
    {"PeriodID": i+1, "Quarter": q, "Year": y} for i, (y, q) in enumerate(periods)
])

# Helper to fetch PeriodID
def get_period_id(quarter, year):
    return time_periods[(time_periods['Quarter'] == quarter) & (time_periods['Year'] == year)].iloc[0]['PeriodID']

# Generate other tables
financial_metrics = []
stock_prices = []
esg_scores = []
employees = []
rnd_spending = []
sector_averages_dict = {}

record_id = price_id = esg_id = emp_id = rnd_id = 1

for _, company in companies.iterrows():
    size_factor = np.random.uniform(0.5, 3.5)
    for _, period in time_periods.iterrows():
        # Metadata
        cid = company.CompanyID
        sector = company.Sector
        period_id = period.PeriodID
        quarter_index = quarters.index(period.Quarter)

        # Financials
        base_rev = {
            'Technology': 150, 'Finance': 120, 'Healthcare': 100, 'Energy': 130,
            'Retail': 80, 'Manufacturing': 90, 'Telecommunications': 110, 'Automotive': 95
        }[sector] * size_factor

        revenue = np.random.normal(loc=base_rev * (1 + 0.05 * ((period.Year - 2020) + quarter_index / 4)), scale=base_rev * 0.1)
        revenue = max(10, revenue)
        op_cost = revenue * np.random.uniform(0.6, 0.85)
        profit = revenue - op_cost
        assets = revenue * np.random.uniform(1.5, 3.5)
        liabilities = assets * np.random.uniform(0.4, 0.8)
        market_cap = revenue * np.random.uniform(2.5, 6.5) / 1000
        stock_price = market_cap * np.random.uniform(15, 25)
        env_score = np.clip(np.random.normal(60 if sector in ['Technology', 'Healthcare'] else 50, 15), 0, 100)
        soc_score = np.clip(np.random.normal(55, 20), 0, 100)
        gov_score = np.clip(np.random.normal(50, 10), 0, 100)
        emp_count = int(revenue * np.random.uniform(10, 40))
        avg_salary = np.random.normal(loc=70, scale=20)
        rnd = revenue * (np.random.uniform(0.08, 0.25) if sector in ['Technology', 'Healthcare', 'Telecommunications'] else np.random.uniform(0.01, 0.1))
        esg_avg = (env_score + soc_score + gov_score) / 3

        # Tables
        financial_metrics.append([record_id, cid, period_id, round(revenue, 2), round(op_cost, 2), round(profit, 2), round(assets, 2), round(liabilities, 2)])
        stock_prices.append([price_id, cid, period_id, round(stock_price, 2), round(market_cap, 2)])
        esg_scores.append([esg_id, cid, period_id, round(env_score, 2), round(soc_score, 2), round(gov_score, 2)])
        employees.append([emp_id, cid, period_id, emp_count, round(avg_salary, 2)])
        rnd_spending.append([rnd_id, cid, period_id, round(rnd, 2)])

        # Sector Averages
        key = (sector, period_id)
        if key not in sector_averages_dict:
            sector_averages_dict[key] = {'revenues': [], 'profits': [], 'mkt_caps': [], 'esg_scores': []}
        sector_averages_dict[key]['revenues'].append(revenue)
        sector_averages_dict[key]['profits'].append(profit)
        sector_averages_dict[key]['mkt_caps'].append(market_cap)
        sector_averages_dict[key]['esg_scores'].append(esg_avg)

        # Increment IDs
        record_id += 1
        price_id += 1
        esg_id += 1
        emp_id += 1
        rnd_id += 1

# 8. Sector Averages Table
sector_averages = []
sector_avg_id = 1
for (sector, period_id), values in sector_averages_dict.items():
    sector_averages.append([
        sector_avg_id, sector, period_id,
        round(np.mean(values['revenues']), 2),
        round(np.mean(values['profits']), 2),
        round(np.mean(values['mkt_caps']), 2),
        round(np.mean(values['esg_scores']), 2)
    ])
    sector_avg_id += 1

# Convert all to DataFrames
financial_df = pd.DataFrame(financial_metrics, columns=['RecordID', 'CompanyID', 'PeriodID', 'Revenue', 'OperatingCost', 'NetProfit', 'Assets', 'Liabilities'])
stock_df = pd.DataFrame(stock_prices, columns=['PriceID', 'CompanyID', 'PeriodID', 'ClosingPrice', 'MarketCap'])
esg_df = pd.DataFrame(esg_scores, columns=['ESGID', 'CompanyID', 'PeriodID', 'EnvironmentalScore', 'SocialScore', 'GovernanceScore'])
employees_df = pd.DataFrame(employees, columns=['EmployeeSnapshotID', 'CompanyID', 'PeriodID', 'EmployeeCount', 'AvgSalary'])
rnd_df = pd.DataFrame(rnd_spending, columns=['RnDID', 'CompanyID', 'PeriodID', 'RnDSpending'])
sector_avg_df = pd.DataFrame(sector_averages, columns=['SectorAverageID', 'Sector', 'PeriodID', 'AvgRevenue', 'AvgProfit', 'AvgMarketCap', 'AvgESGScore'])

# convert to csv
financial_df.to_csv("financial_df.csv",index=False)
stock_df.to_csv("stock_df.csv",index=False)
esg_df.to_csv("esg_df.csv",index=False)
employees_df.to_csv("employees_df.csv",index=False)
rnd_df.to_csv("rnd_df.csv",index=False)
sector_avg_df.to_csv("sector_avg_df.csv",index=False)

# # Show sample from each table
# sample = {
#     "Companies": companies.head(3),
#     "TimePeriods": time_periods.head(3),
#     "FinancialMetrics": financial_df.head(3),
#     "StockPrices": stock_df.head(3),
#     "ESGScores": esg_df.head(3),
#     "Employees": employees_df.head(3),
#     "R&DExpenditure": rnd_df.head(3),
#     "SectorAverages": sector_avg_df.head(3)
# }

# sample
